In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
from sklearn.decomposition import TruncatedSVD
from cuml.neighbors import NearestNeighbors
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
fe = "054"
n_neighbors = 100

In [4]:
if not os.path.exists(f"../output/fe/fe{fe}"):
    os.makedirs(f"../output/fe/fe{fe}")
    os.makedirs(f"../output/fe/fe{fe}/model")

In [5]:
# =========================
# Functions
# =========================
def add_neighbor_name_emb(df, emb):
    dfs = []
    columns = ['id', 'name','latitude','longitude']
    country = df["country"].unique()

    for c in tqdm(country):
        if str(c) == "nan":
            continue
        concat_df = []
        country_df = df[df["country"] == c].reset_index(drop=True)
        country_name_emb = emb[df["country"] == c]
        knn = NearestNeighbors(n_neighbors=min(len(country_df), n_neighbors),metric="cosine")
        knn.fit(country_name_emb)
        dists, nears = knn.kneighbors(country_name_emb)
        targets = country_df[TARGET].values
        for i in range(min(len(country_df), n_neighbors)):
            country_df_ = country_df.copy()
            country_df_["rank"] = i
            country_df_[f"d_near"] = dists[:, i]
            country_df_[f"near_target"] = targets[nears[:, i]]
            for c in columns:
                country_df_[f"near_{c}"] = country_df_[c].values[nears[:, i]]
            concat_df.append(country_df_)
        concat_df = pd.concat(concat_df).reset_index(drop=True)
        dfs.append(concat_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [6]:
# ============================
# Main
# ============================
train = pd.read_csv(TRAIN_PATH)

In [7]:
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(train, train[TARGET],
                                                train[TARGET])):
    train.loc[val_idx, "set"] = i
train["set"].value_counts()

0.0    569406
1.0    569406
Name: set, dtype: int64

In [8]:
train["name"] = train["name"].astype(str)
train["name"] = train["name"].str.lower()

In [9]:
emb = np.load("../output/fe/fe029/model/bert_base_multilingual_embedding.npy")
name = np.load("../output/fe/fe029/model/name.npy",allow_pickle=True)

In [10]:
name_dict = {}
for n,i in enumerate(name):
    name_dict[i] = n

In [11]:
train["name_num"] = train["name"].map(name_dict)

In [12]:
name_emb = np.zeros([len(train),768],dtype=np.float32)
for n,i in enumerate(train["name_num"].values):
    name_emb[n,:] = emb[i]

In [13]:
train_ = add_neighbor_name_emb(train.loc[train["set"] == 0].reset_index(drop=True),name_emb[train["set"] == 0])
val_ = add_neighbor_name_emb(train.loc[train["set"] == 1].reset_index(drop=True),name_emb[train["set"] == 1])

  0%|          | 0/210 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

In [14]:
train = pd.concat([train_, val_]).reset_index(drop=True)
del train_,val_
gc.collect()
train["match"] = train["id"] == train["near_id"]
train["match"] = train["match"].astype(int)
train = train[train["match"] == 0].reset_index(drop=True)

In [16]:
train = reduce_mem_usage(train)
train.to_pickle(f"../output/fe/fe{fe}/model/fe{fe}_{n_neighbors}_name_emb_near.pkl")

Memory usage of dataframe is 15241.55 MB
column =  23
0
Memory usage after optimization is: 15241.55 MB
Decreased by 0.0%
